In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate'

In [4]:
# !pip install smt

In [5]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [6]:
label = "3D_HTTP_swish"
loss_thresh = 20000
x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))


lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [7]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [8]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Sigmoid()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.iter = 0

    
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
       
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)


    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
max_reps = 10
max_iter = 100

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):
    print(label)
    train_loss = []
    test_mse_loss = []
    test_re_loss = []
    beta_val = []


    print(reps)

    torch.manual_seed(reps*36)
    N_D = 5000 #Total number of data points for 'y'
    N_N = 3500
    N_f = 10000 #Total number of collocation points 

    layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')



    nan_flag = train_model(max_iter,reps)


    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    beta_full.append(beta_val)

    if(nan_flag == 1):
        nan_tune.append(tune_reps)
        break

    #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

3D_HTTP_swish
0
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 243755.5 Test MSE 85893.8507646575 Test RE 0.5195416853746291
1 Train Loss 211980.2 Test MSE 66881.09219846984 Test RE 0.458449011406987
2 Train Loss 196338.3 Test MSE 57818.99460653636 Test RE 0.4262600484870081
3 Train Loss 187880.11 Test MSE 50438.01359747613 Test 

96 Train Loss 30521.314 Test MSE 4725.277133036145 Test RE 0.12185770777050342
97 Train Loss 30433.447 Test MSE 4675.946597644751 Test RE 0.12121995919979299
98 Train Loss 30339.951 Test MSE 4651.239589981686 Test RE 0.12089928090014711
99 Train Loss 30277.746 Test MSE 4605.823391209542 Test RE 0.12030758325099535
Training time: 557.76
3D_HTTP_swish
1
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): L

91 Train Loss 24439.656 Test MSE 3961.09131804233 Test RE 0.11156983467103622
92 Train Loss 24164.42 Test MSE 4246.319490845054 Test RE 0.11551694523871636
93 Train Loss 23927.828 Test MSE 4406.0314585092465 Test RE 0.11766929705273439
94 Train Loss 23523.426 Test MSE 4004.058693626725 Test RE 0.1121733214949618
95 Train Loss 23216.43 Test MSE 4171.22803687071 Test RE 0.11449099473361025
96 Train Loss 23113.414 Test MSE 4195.035828719821 Test RE 0.1148172654841287
97 Train Loss 22794.248 Test MSE 4145.231120835026 Test RE 0.11413365813691538
98 Train Loss 22491.873 Test MSE 3869.545024535615 Test RE 0.11027303162489717
99 Train Loss 22107.246 Test MSE 3961.444352360628 Test RE 0.11157480641995389
Training time: 560.82
3D_HTTP_swish
2
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_feat

86 Train Loss 16839.98 Test MSE 1784.1861574755253 Test RE 0.07487889813388048
87 Train Loss 16803.838 Test MSE 1767.2771202493352 Test RE 0.07452323339474878
88 Train Loss 16784.822 Test MSE 1747.9691127096603 Test RE 0.07411502156501158
89 Train Loss 16775.73 Test MSE 1720.307576310621 Test RE 0.07352624943040136
90 Train Loss 16773.258 Test MSE 1718.7057103668503 Test RE 0.07349200944132
91 Train Loss 16770.967 Test MSE 1730.03808059518 Test RE 0.07373389795124952
92 Train Loss 16769.178 Test MSE 1726.8877187246442 Test RE 0.07366673344552728
93 Train Loss 16765.81 Test MSE 1739.0382028048543 Test RE 0.07392544091873
94 Train Loss 16756.156 Test MSE 1762.4164834030335 Test RE 0.07442068021322293
95 Train Loss 16745.28 Test MSE 1763.8860012494642 Test RE 0.07445170004633171
96 Train Loss 16723.168 Test MSE 1849.5754868213035 Test RE 0.07623868427464077
97 Train Loss 16694.77 Test MSE 1789.4293557285996 Test RE 0.07498884094777952
98 Train Loss 16658.229 Test MSE 1784.274653026016 Tes

81 Train Loss 21630.395 Test MSE 2285.1488476098853 Test RE 0.08474159015578203
82 Train Loss 21567.945 Test MSE 2259.5011170127955 Test RE 0.08426469290682094
83 Train Loss 21516.584 Test MSE 2233.4636316890246 Test RE 0.08377777166591667
84 Train Loss 21379.86 Test MSE 2373.439185801001 Test RE 0.08636313846965277
85 Train Loss 21151.367 Test MSE 2255.696286484348 Test RE 0.08419371530043664
86 Train Loss 21051.463 Test MSE 2343.0973329483104 Test RE 0.08580933316902674
87 Train Loss 20942.564 Test MSE 2362.0212541541887 Test RE 0.0861551539496047
88 Train Loss 20620.951 Test MSE 2303.0723464582607 Test RE 0.08507327514061776
89 Train Loss 20501.682 Test MSE 2342.0550504415005 Test RE 0.0857902457171334
90 Train Loss 20177.512 Test MSE 2328.6606446416613 Test RE 0.08554457321632045
91 Train Loss 20083.432 Test MSE 2250.555932046924 Test RE 0.08409772888962473
92 Train Loss 19893.035 Test MSE 2201.173396842136 Test RE 0.0831699594745824
93 Train Loss 19791.23 Test MSE 2157.18647753870

...

72 Train Loss 29325.41 Test MSE 4300.407788393529 Test RE 0.1162503267002111
73 Train Loss 29019.809 Test MSE 4209.833644483513 Test RE 0.11501959379784463
74 Train Loss 28820.459 Test MSE 4165.7289928693735 Test RE 0.11441550153941367
75 Train Loss 28640.434 Test MSE 4052.224443067332 Test RE 0.11284598407150705
76 Train Loss 28449.963 Test MSE 3922.6283015365384 Test RE 0.11102683019114717
77 Train Loss 28303.236 Test MSE 3897.5312416002294 Test RE 0.11067108424107391
78 Train Loss 28070.408 Test MSE 3874.4170610059036 Test RE 0.11034243064006226
79 Train Loss 27673.799 Test MSE 3800.705540135339 Test RE 0.10928774730715098
80 Train Loss 27421.393 Test MSE 3827.5813330752135 Test RE 0.10967346789166718
81 Train Loss 27103.764 Test MSE 3656.786870984029 Test RE 0.10719861805706384
82 Train Loss 26582.557 Test MSE 3734.6630428669732 Test RE 0.10833407368340552
83 Train Loss 25695.178 Test MSE 3412.0074376805323 Test RE 0.10354862748833038
84 Train Loss 25181.238 Test MSE 3185.652805817

67 Train Loss 30413.477 Test MSE 4454.84402509164 Test RE 0.11831930600581879
68 Train Loss 30384.697 Test MSE 4459.098735448659 Test RE 0.11837579442961119
69 Train Loss 30372.484 Test MSE 4431.89264777335 Test RE 0.11801412161053325
70 Train Loss 30354.732 Test MSE 4422.238215391483 Test RE 0.11788551058365367
71 Train Loss 30280.578 Test MSE 4420.72959288547 Test RE 0.11786540086816404
72 Train Loss 30208.188 Test MSE 4443.275453692108 Test RE 0.11816557726461548
73 Train Loss 30151.984 Test MSE 4382.686624630137 Test RE 0.11735715465049938
74 Train Loss 30106.895 Test MSE 4383.227835962386 Test RE 0.11736440055543566
75 Train Loss 30034.357 Test MSE 4322.583365543847 Test RE 0.1165496707879132
76 Train Loss 29965.047 Test MSE 4346.559262152295 Test RE 0.11687245446348976
77 Train Loss 29911.213 Test MSE 4293.597876281335 Test RE 0.11615824611141677
78 Train Loss 29840.738 Test MSE 4289.184365743479 Test RE 0.11609852959025226
79 Train Loss 29759.203 Test MSE 4308.777494484276 Test 

62 Train Loss 17589.65 Test MSE 1603.9894844725302 Test RE 0.0709970200643028
63 Train Loss 17540.188 Test MSE 1620.2326838761246 Test RE 0.07135559905308801
64 Train Loss 17384.197 Test MSE 1645.1910764166666 Test RE 0.07190308676046617
65 Train Loss 17196.201 Test MSE 1690.5811603480088 Test RE 0.07288822506841375
66 Train Loss 17138.428 Test MSE 1680.4705368632851 Test RE 0.07266994198289393
67 Train Loss 17110.477 Test MSE 1708.9085862699915 Test RE 0.07328224713750192
68 Train Loss 17070.574 Test MSE 1770.889913552457 Test RE 0.07459936734611157
69 Train Loss 17019.033 Test MSE 1918.129307273784 Test RE 0.07763870855533166
70 Train Loss 16945.932 Test MSE 1866.8619405492575 Test RE 0.07659412569388162
71 Train Loss 16859.44 Test MSE 1878.0392615237615 Test RE 0.07682307661136058
72 Train Loss 16826.195 Test MSE 1805.411572200705 Test RE 0.0753229764384722
73 Train Loss 16814.842 Test MSE 1808.4003906450243 Test RE 0.07538529840669123
74 Train Loss 16738.654 Test MSE 1774.815701244

57 Train Loss 19498.742 Test MSE 1990.9829235110149 Test RE 0.07909938917319354
58 Train Loss 19383.535 Test MSE 2115.9276957517063 Test RE 0.08154357989834204
59 Train Loss 19181.053 Test MSE 2021.6506531680454 Test RE 0.07970625741403442
60 Train Loss 18992.723 Test MSE 1959.0114895848642 Test RE 0.07846172532575287
61 Train Loss 18897.95 Test MSE 1854.739252858708 Test RE 0.07634503417486874
62 Train Loss 18690.541 Test MSE 1804.5823611079902 Test RE 0.07530567683158543
63 Train Loss 18577.15 Test MSE 1797.2193579766658 Test RE 0.07515188981356868
64 Train Loss 18541.379 Test MSE 1798.4869155713995 Test RE 0.07517838701266297
65 Train Loss 18503.084 Test MSE 1831.9460486089968 Test RE 0.0758744754958606
66 Train Loss 18241.5 Test MSE 1892.4823426831397 Test RE 0.07711791518113247
67 Train Loss 18077.023 Test MSE 1964.8507801320156 Test RE 0.07857857505010231
68 Train Loss 18002.598 Test MSE 1846.8537625249714 Test RE 0.0761825694868437
69 Train Loss 17927.816 Test MSE 1888.224495911

52 Train Loss 20511.777 Test MSE 2184.3202390329134 Test RE 0.08285095467962805
53 Train Loss 20492.91 Test MSE 2208.7407973013455 Test RE 0.08331280155656658
54 Train Loss 20368.914 Test MSE 2310.595013938009 Test RE 0.08521210191545926
55 Train Loss 20317.45 Test MSE 2328.8117879351444 Test RE 0.08554734933545415
56 Train Loss 20216.559 Test MSE 2159.2084141250048 Test RE 0.08237333401554331
57 Train Loss 20078.422 Test MSE 2072.1410390468754 Test RE 0.08069544450986724
58 Train Loss 19791.29 Test MSE 2053.3690306175877 Test RE 0.08032909349144701
59 Train Loss 19592.025 Test MSE 1808.8945699671629 Test RE 0.07539559792640146
60 Train Loss 19507.629 Test MSE 1883.7552614446274 Test RE 0.07693989714046544
61 Train Loss 19269.836 Test MSE 1883.4080237829621 Test RE 0.07693280554480808
62 Train Loss 18704.24 Test MSE 1839.2771654599503 Test RE 0.07602614187800832
63 Train Loss 18235.424 Test MSE 1891.630960766318 Test RE 0.07710056648968658
64 Train Loss 18093.193 Test MSE 1757.90047330